In [2]:
from urllib.request import Request
from urllib.request import urlopen
from bs4 import BeautifulSoup
import time
import pandas as pd

In [3]:
#function for adding urls of games with at least 2 offers, from "last activity" pages
def getlist(url):
    user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
    headers={'User-Agent':user_agent,}
    
    request=Request(url,None,headers) #The assembled request
    response = urlopen(request)
    bs = BeautifulSoup(response.read(), 'html.parser')
    
    #creating list of all games on single page using BeautifulSoup syntax
    games_list = bs.find_all('div', {'class':'col-md-12 col-xs-12 col-sm-12 item'})
    
    url_list = []
    for game in games_list:
        #iterating over the list of games to get all relevant info using BeautifulSoup syntax
        offers = game.find('div', {'class':'col-md-2 col-sm-2 col-xs-2 text-right oswald pc'}).find('p', {'class':'prc-text'}).get_text()
        offers = float(offers.replace("ofert", "").replace("y", "").replace("a", "").replace("od", "").replace(" ", ""))
        #if game has 2 or more active offers adding the individual offer link to a list
        if offers > 1:
            link = 'https://bazar.lowcygier.pl' + game.find('div', {'class':'col-md-7 col-sm-4 col-xs-6 nopadding'}).h4.a['href']
            if link not in url_list:
                url_list.append(link)

    return url_list

In [4]:
#iterating over "last activity" pages to get urls to 400 (or more) recently modified offers
links_list = []
for i in range(1, 5):
    links_list += getlist('https://bazar.lowcygier.pl/?options=&type%5B0%5D=sell&platform%5B0%5D=1&payment%5B0%5D=1&game_type=&game_genre=&title=&game_id=&sort=-last_activity&per-page=100&page=' + str(i))
    time.sleep(1)

In [5]:
#function to get the relevant data - game name, lowest offer, second lowest offer
def getinfo(url):
    user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
    headers={'User-Agent':user_agent,} 

    request=Request(url,None,headers) #The assembled request
    response = urlopen(request)
    bs = BeautifulSoup(response.read(), 'html.parser')
    
    #get all relevant info about single offer using BeautifulSoup syntax
    name = bs.find('h2', {'class':'test'}).get_text()
    lowest_offer = float(bs.find('div', {'class':'price'}).get_text().replace(" ", "").replace("\n", "").replace("zł", "").replace(",", "."))
    second_offer = float(bs.find_all('p', {'class':'prc text-right'})[0].get_text().replace(" ", "").replace("\n", "").replace("zł", "").replace(",", "."))
    info = [name, lowest_offer, second_offer, (second_offer - lowest_offer)]
    time.sleep(1) #let's not kill the server
    return info

In [6]:
#iterating over list of urls to gather relevant data about all the games and put it in one place
data = []
number = 0
for link in links_list:
    data.append(getinfo(link))
    #code below is just to add some visual feedback during the data gathering process
    number += 1
    if number % 10 == 0:
        print(number, "/", len(links_list))
    elif number == len(links_list):
        print("Completed!")

10 / 334
20 / 334
30 / 334
40 / 334
50 / 334
60 / 334
70 / 334
80 / 334
90 / 334
100 / 334
110 / 334
120 / 334
130 / 334
140 / 334
150 / 334
160 / 334
170 / 334
180 / 334
190 / 334
200 / 334
210 / 334
220 / 334
230 / 334
240 / 334
250 / 334
260 / 334
270 / 334
280 / 334
290 / 334
300 / 334
310 / 334
320 / 334
330 / 334
Completed!


In [7]:
df = pd.DataFrame(data, columns = ['Game', 'Lowest Offer', 'Second Offer', 'Price Difference'])

In [9]:
df.describe()

,Lowest Offer,Second Offer,Price Difference
count,334.000000,334.000000,334.000000
mean,5.045150,5.585359,0.540210
std,7.267039,7.877910,2.010902
min,0.500000,0.500000,0.000000
25%,0.900000,1.000000,0.010000
50%,2.175000,2.500000,0.090000
75%,5.997500,6.967500,0.397500
max,40.000000,41.740000,32.000000


In [10]:
df.to_csv('recent_games.csv')

In [12]:
df.head(10)

,Game,Lowest Offer,Second Offer,Price Difference
0,Double Dragon Trilogy,0.75,0.80,0.05
1,Little Big Adventure 2,0.90,0.98,0.08
2,Little Big Adventure - Enhanced Edition,1.99,2.00,0.01
3,Wandersong,1.00,1.19,0.19
4,Vikings - Wolves of Midgard,9.88,9.89,0.01
5,Finding Paradise,3.00,3.35,0.35
6,Karateka,0.65,0.67,0.02
7,Monster Prom,0.50,0.95,0.45
8,The Journey Down: Chapter Three,1.00,1.30,0.30
9,I am not a Monster,0.50,0.50,0.00
